# Making masks

In [15]:
%matplotlib inline
%matplotlib widget
#try widget instead if graphs aren't showing

import os
import cv2
import maxflow
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import ipywidgets as widgets
from matplotlib.patches import Circle
from IPython.display import display

In [16]:
def graphcut(image, scribble_mask, lambda_val=50, sigma=10):
    H, W, _ = image.shape
    img = image.astype(np.float64)
    
    # identify scribbled regions
    fg_pixels = scribble_mask == 1  # definite foreground
    bg_pixels = scribble_mask == 2  # definite background
    
    # compute mean colors for scribbled foreground and background
    fg_mean = np.mean(img[fg_pixels], axis=0) if np.any(fg_pixels) else np.zeros(3)
    bg_mean = np.mean(img[bg_pixels], axis=0) if np.any(bg_pixels) else np.zeros(3)
    
    # data costs based on squared color distances
    data_cost_fg = np.sum((img - fg_mean)**2, axis=2)
    data_cost_bg = np.sum((img - bg_mean)**2, axis=2)
    
    # normalize the data costs
    data_cost_fg = data_cost_fg / (np.max(data_cost_fg) + 1e-8)
    data_cost_bg = data_cost_bg / (np.max(data_cost_bg) + 1e-8)
    
    # build the graph
    g = maxflow.Graph[float]()
    nodeids = g.add_grid_nodes((H, W))
    
    # terminal (data) capacities
    source_cap = data_cost_fg.copy()
    sink_cap = data_cost_bg.copy()
    
    # for scribbled pixels, force the label
    source_cap[scribble_mask == 1] = 0      # Force foreground.
    sink_cap[scribble_mask == 1] = 1e9
    source_cap[scribble_mask == 2] = 1e9      # Force background.
    sink_cap[scribble_mask == 2] = 0
    
    g.add_grid_tedges(nodeids, source_cap, sink_cap)
    
    # add smoothness (neighborhood) terms
    for i in range(H):
        for j in range(W):
            if j < W - 1:
                diff = np.linalg.norm(img[i, j] - img[i, j+1])
                weight = lambda_val * np.exp(- (diff**2) / (2 * sigma**2))
                g.add_edge(nodeids[i, j], nodeids[i, j+1], weight, weight)
                
            if i < H - 1:
                diff = np.linalg.norm(img[i, j] - img[i+1, j])
                weight = lambda_val * np.exp(- (diff**2) / (2 * sigma**2))
                g.add_edge(nodeids[i, j], nodeids[i+1, j], weight, weight)
                
    # compute the min-cut
    g.maxflow()
    seg_mask = np.zeros((H, W), dtype=bool)
    for i in range(H):
        for j in range(W):
            seg_mask[i, j] = True if g.get_segment(nodeids[i, j]) == 0 else False
            
    return seg_mask